In [1]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [24]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [275]:
# skip if result.txt exists
y16 = glob.glob('data/2016/*.txt')
y15 = glob.glob('data/2015/*.txt')
y14 = glob.glob('data/2014/*.txt')
y14_16=y14+y15+y16
with open("data/data.txt", "wb") as outfile:
    for f in y14_16:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==10:
                    outfile.write(line+'\n')

In [276]:
df=pd.read_table('data/data.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author','lic']
df.shape

(2995098, 10)

In [277]:
#df=df[df['author']==1]
#df.shape

In [278]:
df['year']=df['year'].astype(int)

In [279]:
df=df[df['year']<=2016][df['year']>=2014]
df.shape

C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


(2970751, 10)

In [280]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [281]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(2970239, 13)

In [282]:
df=df[['year','count','citation','issnp','issne']].fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [283]:
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [284]:
s=df.issn.str.split(',',expand=True).stack()

In [285]:
i = s.index.get_level_values(0)

In [286]:
df2 = df.loc[i].copy()

In [287]:
df2["@id"] = s.values

In [314]:
dg=df2.groupby(['@id','year']).agg({'count':sum,'citation':sum}).reset_index()
dg['impact']=dg['citation'].div(dg['count']).round(2)
dg.shape

(57335, 5)

In [348]:
dg_h=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()
dg_h['h']=dg_h.citation.apply(lambda x: h_(x))
dg_h.shape

(48610, 3)

In [349]:
dg_i=df2.groupby(['@id']).agg({'count':sum,'citation':sum}).reset_index()
dg_i['impact']=dg_i['citation'].div(dg_i['count']).round(2)
dg_i.shape

(48610, 4)

In [350]:
dg_flat=dg.pivot(index='@id',columns='year')
dg_flat.columns = [col[0]+'_'+str(col[1])[3] for col in dg_flat.columns.values]
dg_flat=dg_flat.reset_index()

In [356]:
dg_merge=dg_flat.merge(dg_i[['@id','count','citation','impact']])\
.merge(dg_h[['@id','h']]).fillna(0,downcast='infer')
dg_merge.shape

(48610, 14)

In [357]:
dic=[]
for index, i in dg_merge[:].iterrows():
    journal={}
    journal['@id']=i['@id']
    #issn_list=[i['@id']]
    #if (i['issne2']!='') and (i['issne2']!=i['@id']):
        #issn_list.append(i['issne2'])
    #journal['e:issn']=issn_list
    journal['e:count_4']=i['count_4']
    journal['e:citation_4']=i['citation_4']
    journal['e:impact_4']=round(i['impact_4'],2)
    journal['e:count_5']=i['count_5']
    journal['e:citation_5']=i['citation_5']
    journal['e:impact_5']=round(i['impact_5'],2)
    journal['e:count_6']=i['count_6']
    journal['e:citation_6']=i['citation_6']
    journal['e:impact_6']=round(i['impact_6'],2)
    journal['e:count']=i['count']
    journal['e:citation']=i['citation']
    journal['e:impact']=round(i['impact'],2)
    journal['e:h']=i['h']
    if journal not in dic:
        dic.append(journal)

In [358]:
with open('journals_stats.jsonld', 'w') as outfile:
    json.dump(dic, outfile)

In [354]:
dg_h[dg_h['@id']=='1225-1518']

,@id,citation,h
12117,1225-1518,"(0, 0, 0, 0, 0, 0, 0, 0)",0
